In [1]:
%pip install opencv-python numpy albumentations scikit-learn


     -------------------------------------- 216.5/216.5 kB 1.9 MB/s eta 0:00:00
     --------------------------------------- 12.8/12.8 MB 10.2 MB/s eta 0:00:00
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl (161 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
     ---------------------------------------- 38.8/38.8 MB 5.6 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached pydantic_core-2.23.4-cp311-none-win_amd64.whl (1.9 MB)
     -------------------------------------- 315.4/315.4 kB 6.5 MB/s eta 0:00:00
     -------------------------------------- 228.2/228.2 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.75.0 requires pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,<2.0.0,>=1.6.2, but you have pydantic 2.9.2 which is incompatible.

[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import albumentations as A
from glob import glob
from sklearn.model_selection import train_test_split

# Apply CLAHE preprocessing
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    if len(image.shape) == 3 and image.shape[2] == 3:
        # Apply CLAHE to each channel for color images
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return clahe.apply(image)

# Normalize images between 0 and 1
def normalize(image):
    image = image.astype(np.float32) / 255.0
    return image

# Define data augmentation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ElasticTransform(p=0.5)
])

# Load and preprocess images
def load_images(image_paths):
    images = []
    for path in image_paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        image = apply_clahe(image)
        image = normalize(image)
        augmented = transform(image=image)
        images.append(augmented['image'])
    return np.array(images)

# Split dataset into training and test sets (80-20 split)
image_paths = glob('Data/Data/*/*.tif')
mask_paths = glob('Data/Data/*/*_mask.tif')
image_paths = [i for i in image_paths if i not in mask_paths]

train_images, test_images, train_masks, test_masks = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)


In [3]:
%pip install -U segmentation_models_pytorch



     -------------------------------------- 109.5/109.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 436.4/436.4 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 11.6 MB/s eta 0:00:00
     -------------------------------------- 286.0/286.0 kB 8.6 MB/s eta 0:00:00
     -------------------------------------- 199.4/199.4 MB 1.8 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16464 sha256=74f6757730ea847ede070fe3d73db3d3b490864d6d82ec8d2d80c503512503cf
  Stored in directory: c:\users\karan\appdata\local\pip\cac


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import segmentation_models_pytorch as smp

# Define U-Net++ model with ResNet backbone
unet_plus_plus_model = smp.UnetPlusPlus(
    encoder_name="resnet34",        # Use a pretrained ResNet34 encoder
    encoder_weights="imagenet",     # Use imagenet pretrained weights
    in_channels=1,                  # For grayscale MRI images
    classes=1,                      # Output is binary segmentation (0 or 1 for metastasis)
    activation=None                 # Use no activation as we will apply Sigmoid/Softmax later
)

# Summary of the model
print(unet_plus_plus_model)


UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [5]:
%pip install keras


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install 


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, UpSampling2D, Concatenate, Activation, Multiply, MaxPooling2D

def attention_gate(x, g, inter_channel):
    # Downsample x
    theta_x = Conv2D(inter_channel, (2, 2), strides=(2, 2), padding='same')(x)
    
    # Ensure g is downsampled to match the shape of theta_x
    phi_g = Conv2D(inter_channel, (1, 1), padding='same')(g)
    # Add downsampling to g to match theta_x dimensions
    phi_g = MaxPooling2D(pool_size=(2, 2))(phi_g)

    # Add and apply activation
    add = Activation('relu')(theta_x + phi_g)
    psi = Conv2D(1, (1, 1), padding='same')(add)
    sigmoid = Activation('sigmoid')(psi)
    
    # Multiply the original input with the attention weights
    x = Multiply()([x, sigmoid])
    return x

def attention_unet(input_shape=(256, 256, 1)):
    inputs = Input(input_shape)
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    
    # Assuming you want to maintain a reference to the encoder output
    attention1 = attention_gate(conv1, conv1, 64)

    # Final output layer, adjust as needed for your application
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(attention1)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

model = attention_unet()
model.summary()


In [ ]:
model.save('unet.h5')

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import torch.nn.functional as F

import cv2  # Ensure you have cv2 to load images

import numpy as np

class MRIDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images  # Expecting file paths
        self.masks = masks    # Expecting file paths

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Load the image and mask
        image = cv2.imread(self.images[idx], cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        mask = cv2.imread(self.masks[idx], cv2.IMREAD_GRAYSCALE)    # Load as grayscale
        
        if image is None or mask is None:
            raise ValueError(f"Image or mask at index {idx} could not be loaded.")

        # Ensure they are numpy arrays
        image = image.astype(np.float32)  # Ensure it's float32
        mask = mask.astype(np.float32)    # Ensure it's float32

        # Add a channel dimension
        image = np.expand_dims(image, axis=0)  # shape: (1, height, width)
        mask = np.expand_dims(mask, axis=0)    # shape: (1, height, width)

        return torch.tensor(image, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)



# DICE Score Calculation
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    return 1 - (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# Training loop
def train_model(model, train_loader, test_loader, epochs=20, lr=1e-4):
    optimizer = Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        for images, masks in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = dice_loss(outputs, masks)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch}/{epochs}, Loss: {loss.item()}')

# Prepare data loaders
train_dataset = MRIDataset(train_images, train_masks)
test_dataset = MRIDataset(test_images, test_masks)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# Train Nested U-Net
train_model(unet_plus_plus_model, train_loader, test_loader)


In [ ]:
torch.save(unet_plus_plus_model,'unet_model.pth')